<a href="https://colab.research.google.com/github/caiocesarcosta/ai-job-assistant/blob/main/ai_job_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade transformers

import json
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer, TFBertModel, BertConfig
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Carregando os Dados Profissionais ---
with open("dados_profissionais.json", "r") as arquivo:
    dados = json.load(arquivo)

# --- Carregando as Perguntas e Respostas do Arquivo ---
with open("perguntas_respostas.json", "r") as arquivo:
    perguntas_respostas = json.load(arquivo)

# --- Pré-processamento dos Dados com BERT ---
perguntas = list(perguntas_respostas.keys())
respostas = list(perguntas_respostas.values())

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

perguntas_tokenizadas = [tokenizer.encode(pergunta, add_special_tokens=True) for pergunta in perguntas]
perguntas_treino = pad_sequences(perguntas_tokenizadas, padding='post', maxlen=128)  # Ajuste maxlen se necessário

# --- Carregando a Configuração do BERT ---
config = BertConfig.from_pretrained('bert-base-uncased')

# --- Criando o Modelo com BERT e Fine-tuning ---
modelo_bert = TFBertModel.from_pretrained('bert-base-uncased', config=config)  # Use TFBertModel

modelo_classificacao = tf.keras.Sequential([
    modelo_bert,
    tf.keras.layers.Dense(len(perguntas), activation='softmax')
])

# --- Compilando o Modelo ---
modelo_classificacao.compile(optimizer='adam',
                             loss='sparse_categorical_crossentropy',
                             metrics=['accuracy'])

# --- Criando Rótulos para as Perguntas ---
rotulos_perguntas = np.arange(len(perguntas))

# --- Treinando o Modelo ---
modelo_classificacao.fit(perguntas_treino, rotulos_perguntas, epochs=3)  # Ajuste o número de épocas

# --- Função para Encontrar a Resposta ---
def encontrar_resposta(pergunta):
    pergunta_tokenizada = tokenizer.encode(pergunta, add_special_tokens=True)
    pergunta_processada = pad_sequences([pergunta_tokenizada], padding='post', maxlen=128)
    previsao = modelo_classificacao.predict(pergunta_processada)
    indice_resposta = np.argmax(previsao[0])
    return perguntas_respostas[perguntas[indice_resposta]]

# --- Testando o Chatbot ---
while True:
    pergunta = input("Recrutador: ")
    if pergunta == "sair":
        break
    resposta = encontrar_resposta(pergunta)
    print("Você:", resposta)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/sequential.py", line 344, in _build_graph_network_for_inferred_shape
        raise ValueError(SINGLE_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')